In [79]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os

### Definindo funções comuns

In [81]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
def extrair_links(html, lista):
    soup = BeautifulSoup(html, 'html.parser')
    produtos = soup.find_all('div', class_='produto')
    
    for produto in produtos:
        link = produto.find('a', href=True)

        s_desc = None
        c_desc = None

        try:
            preco_div = produto.find('div', {'data-target': 'preco-produto'})

            # Verificar o tipo de estrutura de preço
            if preco_div.find('p', {'class': 'font-weight-bold'}):  # Caso com desconto
                c_desc = preco_div.find('p', {'class': 'font-weight-bold'}).text
                s_desc = preco_div.find('p', {'class': 'text-muted text-weight-bold'}).text

            elif preco_div.find('span', {'class': 'font-weight-bold font-xl'}):  # Caso x produtos por valor y
                quantidade_x_element = preco_div.find('span', {'class': 'font-weight-bold'})
                preco_x_element = preco_div.find('span', {'class': 'font-weight-bolder text-marine font-xl'})
                preco_y_element = preco_div.find('p').find('span', {'class': 'text-muted small'})

                s_desc = quantidade_x_element.text if quantidade_x_element else ''
                c_desc = s_desc

            elif preco_div.find('span', {'class': 'font-weight-bold font-2xl'}):  # Caso sem desconto
                s_desc = preco_div.find('span', {'class': 'font-weight-bold font-2xl'}).text
                c_desc = s_desc

        except AttributeError as e:
            print(f'Erro ao extrair informações do produto {link}: {e}')
            
        resultado = {'URL': 'https://farmaciapermanente.com.br' + link['href'], 'Preço sem desconto': s_desc, 'Preço com desconto': c_desc}

        lista.append(resultado)
    return lista

def extrair_infos(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        nome_elemento = soup.find('h1', {'data-target': 'nome_produto'})
        ean_elemento = soup.find('p', class_='text-muted my-2 font-sm font-08rem mb-3')
        marca_elemento = soup.find('p', class_='text-muted my-2 font-sm font-08rem mb-3')
        descricao_elemento = soup.find('div', {'id': 'tabCollapse-descricao'})

        nome = nome_elemento.get_text(strip=True) if nome_elemento else None
        ean = ean_elemento.get_text(strip=True).split('EAN: ')[1] if ean_elemento else None
        marca = marca_elemento.find('a').get_text(strip=True) if marca_elemento else None
        descricao = descricao_elemento.get_text(strip=True) if descricao_elemento else None

        informacoes = {
            'Produto': nome,
            'EAN': ean,
            'Marca': marca,
            'Descrição': descricao,
        }
        return informacoes

    except Exception as e:
        print(f"Erro ao processar a URL {url}: {e}")
        return None
  
def process_url(url):
    info = extrair_infos(url)
    if info:
        return info

A primeira aba do código trabalha na extração dos links e preços dos produtos por categorias, e posteriormente acessamos cada link para extrair o restante das informações.

### Extraindo links

In [85]:
categorias = ['medicamentos', 'saude', 'beleza', 'vitaminas-e-suplementos', 'conveniencia', 'dermocosmeticos', 'mamaes-e-bebes', 'cuidados-diarios']

driver = webdriver.Chrome()
links = []

for categoria in categorias:
    ref = f"https://www.farmaciapermanente.com.br/categorias/{categoria}"
    driver.get(ref)
    wait = WebDriverWait(driver, 10)

    while True:
        try:
            wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.load.load-body')))
            extrair_links(driver.page_source, links)

            # Encontrar e clicar no botão de próxima página
            next_page_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-pagina="next"]')))
            next_page_button.click()
            time.sleep(2)

        except (StaleElementReferenceException, TimeoutException):
            break

driver.quit()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175


### Extraindo informações

In [86]:
# Criando df com links e salvando arquivo
df_links = pd.DataFrame(links)
df_links = df_links.drop_duplicates()
df_links.to_csv('df_links.csv', index=False)

In [90]:
# Visitaremos cada link e puxaremos as informações desejadas do html
max_workes = os.cpu_count()

infos = []

with ThreadPoolExecutor(max_workers=max_workes) as executor:
    # Use the map function to apply the process_url function to each URL in parallel
    results = list(tqdm(executor.map(process_url, df_links['URL']), total=len(df_links)))

# Collect the results
infos = list(filter(None, results))

 32%|███▏      | 3241/10206 [15:19<37:22,  3.11it/s]  

Erro ao processar a URL https://farmaciapermanente.com.br/ad-til-takeda-caixa-20ml-solucao-oral: list index out of range


100%|██████████| 10206/10206 [46:59<00:00,  3.62it/s] 


### Tratamento de dados para output final

In [92]:
df_infos = pd.DataFrame(infos)
df_infos.to_csv('df_infos.csv')

In [100]:
# Algumas linhas retornaram erros para o dataframe de informações no momento da extração, então essas serão removidas do dataframe de links para termos o mesmo número de linhas
links_a_remover = ['https://farmaciapermanente.com.br/ad-til-takeda-caixa-20ml-solucao-oral']
df = df_links[~df_links['URL'].isin(links_a_remover)]
df = df.reset_index(drop=True)

# Juntando os dois dataframes
df_final = pd.concat([df_infos, df.iloc[:, -2:]], axis=1)

In [101]:
# Formatando as colunas de preços e adicionando a % de desconto
df_final['Preço sem desconto'] = df_final['Preço sem desconto'].replace('[^\d,]', '', regex=True).str.replace(',', '.').astype(float)
df_final['Preço com desconto'] = df_final['Preço com desconto'].replace('[^\d,]', '', regex=True).str.replace(',', '.').astype(float)

# Fill NaN values in 'Preço sem desconto' with 'Preço com desconto'
df_final['Preço sem desconto'].fillna(df_final['Preço com desconto'], inplace=True)

# Calculate '% de desconto'
df_final['% de desconto'] = df_final.apply(
    lambda row: "{:.1f}%".format(round((row['Preço sem desconto'] - row['Preço com desconto']) / row['Preço sem desconto'] * 100))
    if pd.notna(row['Preço sem desconto']) and pd.notna(row['Preço com desconto']) and row['Preço sem desconto'] != row['Preço com desconto']
    else None,
    axis=1
)

In [102]:
# Adicionando colunas finais
df_final['Farmacia'] = 'Santo Remedio'
df_final['Região'] = 'Sudeste'
df_final['Cidade'] = 'Sao Paulo'

In [103]:
df_final.to_csv('df_final.csv', index=False)